In [1]:
def get_nxos_cam_table(username,
                       password,
                       host_group,
                       play_path,
                       private_data_dir='.',
                       interface=None):
    '''
    Gets the NXOS CAM table and adds the vendor OUI.

    Args:
        username (str):         The username to login to devices
        password (str):         The password to login to devices
        host_group (str):       The inventory host group
        play_path (str):        The path to the playbooks directory
        private_data_dir (str): The path to the Ansible private data directory
        interface (str):        The interface (defaults to all interfaces)

    Returns:
        df_cam (DataFrame):     The CAM table and vendor OUI
    '''
    if interface:
        cmd = f'show mac address-table interface {interface}'
    else:
        cmd = 'show mac address-table'
    extravars = {'username': username,
                 'password': password,
                 'host_group': host_group,
                 'commands': cmd}
 
    # Execute the pre-checks
    playbook = f'{play_path}/cisco_nxos_run_commands.yml'
    runner = ansible_runner.run(private_data_dir=private_data_dir,
                                playbook=playbook,
                                extravars=extravars)
    
    # Parse the output and add it to 'data'
    data = dict()
    data['device'] = list()
    data['mac'] = list()
    data['interface'] = list()
    data['vendor'] = list()
    data['vlan'] = list()
    for event in runner.events:
        if event['event'] == 'runner_on_ok':
            event_data = event['event_data']
            
            device = event_data['remote_addr']
            
            output = event_data['res']['stdout'][0].split('\n')
            output = output[5:]
            for line in output:
                mac = line.split()[2]
                interface = line.split()[-1]
                vlan = line.split()[1]
                try:
                    vendor = find_mac(mac)
                except Exception:
                    vendor = 'unknown'
                data['device'].append(device)
                data['mac'].append(mac)
                data['interface'].append(interface)
                data['vendor'].append(vendor)
                data['vlan'].append(vlan)
    df_cam = pd.DataFrame.from_dict(data)
    cols = ['device', 'mac', 'vlan', 'interface', 'vendor']
    df_cam = df_cam[cols]

    return df_cam